In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/dataset/images.zip", 'r')
zip_ref.extractall("/content/data")
zip_ref.close()

In [0]:
#https://medium.com/@shivamrawat_756/how-to-prevent-google-colab-from-disconnecting-717b88a128c0

In [0]:
from keras.callbacks import ModelCheckpoint
#import libraries
#import keras
import numpy as np
from keras.applications import vgg16
#from keras.preprocessing.image import load_img
#from keras.preprocessing.image import img_to_array
#from keras.applications.imagenet_utils import decode_predictions
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
!pip install keras_vggface

In [0]:
#Load the VGG model
image_size=224
#vgg_model = vgg16.VGG16(weights='imagenet')
from keras_vggface.vggface import VGGFace

vgg_conv = VGGFace(weights='vggface', include_top=False, input_shape=(image_size, image_size, 3))





58916864/58909280 [==============================] - 4s 0us/step




In [0]:
# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers:
    layer.trainable = False

In [0]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(7, activation='softmax'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.summary()    


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vggface_vgg16 (Model)        (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               12845568  
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 455       
Total params: 27,593,543
Trainable params: 12,878,855
Non-trainable params: 14,714,688
_________________________________

In [0]:
from keras.preprocessing.image import ImageDataGenerator
image_gen = ImageDataGenerator(rotation_range=30, # rotate the image 30 degrees
                               width_shift_range=0.1, # Shift the pic width by a max of 10%
                               height_shift_range=0.1, # Shift the pic height by a max of 10%
                               rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.2, # Shear means cutting away part of the image (max 20%)
                               zoom_range=0.2, # Zoom in by 20% max
                               horizontal_flip=True, # Allo horizontal flipping
                               fill_mode='nearest' # Fill in missing pixels with the nearest filled value
                              )

In [0]:
# Data Generator for Training data
train_generator = image_gen.flow_from_directory('/content/data/images/train/',
        target_size=(image_size, image_size),
        batch_size=80,
        class_mode='categorical')


Found 28821 images belonging to 7 classes.


In [0]:
validation_generator = image_gen.flow_from_directory('/content/data/images/validation',
        target_size=(image_size, image_size),
        batch_size=30,
        class_mode='categorical',
        shuffle=False)

Found 7066 images belonging to 7 classes.


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=0.001),
              metrics=['acc'])

In [0]:
checkpoint = ModelCheckpoint('/content/data/vggface_weights.best.hdf5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [0]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      callbacks=callbacks_list,
      verbose=2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/30
 - 358s - loss: 1.3867 - acc: 0.4773 - val_loss: 1.1903 - val_acc: 0.5467

Epoch 00001: val_loss improved from inf to 1.19028, saving model to /content/data/vggface_weights.best.hdf5
Epoch 2/30
 - 342s - loss: 1.2316 - acc: 0.5366 - val_loss: 1.1290 - val_acc: 0.5801

Epoch 00002: val_loss improved from 1.19028 to 1.12900, saving model to /content/data/vggface_weights.best.hdf5
Epoch 3/30
 - 339s - loss: 1.1838 - acc: 0.5586 - val_loss: 1.1179 - val_acc: 0.5828

Epoch 00003: val_loss improved from 1.12900 to 1.11791, saving model to /content/data/vggface_weights.best.hdf5
Epoch 4/30
 - 337s - loss: 1.1610 - acc: 0.5672 - val_loss: 1.1099 - val_acc: 0.5872

Epoch 00004: val_loss improved from 1.11791 to 1.10989, saving model to /content/data/vggface_weights.best.hdf5
Epoch 5/30
 - 341s - loss: 1.1375 - acc: 0.5758 - val_loss: 1.0972 - val_acc: 0.5961

Epoch 00005: val_loss improved f